# Multi-Learn Dreambooth

This tool is meant for automation of creating models of a person or multiple people
NOTE: This does NOT create a single model that knows all faces, but seperate models where ONE person is known
This tool simply make the process of making multiple models a lot easier.

In [3]:
models_to_train = [
    "sterdekie",
    "aquaFPS"
]

In [4]:
import builtins

for new_model in models_to_train:
    builtins.word_to_train = new_model
    project_name = new_model

    # MAX STEPS
    # It's how long you want your training to go.
    # If you're seeing this message, I'm literally at my computer right now fixing this up:
    max_training_steps = 1000

    class_word = "person"  # << match this word to the class word from regularization images above
    reg_data_root = "/workspace/Dreambooth-Stable-Diffusion/outputs/txt2img-samples/samples/person_ddim"

    !python "main.py" \
        --base configs/stable-diffusion/v1-finetune_unfrozen.yaml \
        -t \
        --actual_resume "model.ckpt" \
        --reg_data_root {reg_data_root} \
        -n {project_name} \
        --gpus 0, \
        --data_root "/workspace/Dreambooth-Stable-Diffusion/training_images/" + {new_model} \
        --max_training_steps {max_training_steps} \
        --class_word class_word \
        --no-test
    
    directory_paths = !ls -d logs/*
    last_checkpoint_file = directory_paths[-1] + "/checkpoints/last.ckpt"

    !python "prune_ckpt.py" --ckpt {last_checkpoint_file}

    last_checkpoint_file_pruned = directory_paths[-1] + "/checkpoints/last-pruned.ckpt"
    training_samples = !ls training_samples
    date_string = !date +"%Y-%m-%dT%H-%M-%S"
    
    file_name = date_string[-1] + "_" + project_name + "_" + str(len(training_samples)) + "_training_images_" +  str(max_training_steps) + "_max_training_steps_" + class_word + "_class_word.ckpt"
    !mkdir -p trained_models
    !mv {last_checkpoint_file_pruned} trained_models/{file_name}

10
